In [1]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow import keras
import seaborn as sbn

In [2]:
# import data TODO: Change the file path to the dataset this question needed
df = pd.read_csv('..\\source_data\\crude_co2_p1_df.csv')
df

,week,crude_oil,cpi,coal_consumption,ngas_consumption,us_pop,oil_production,tree_cover_loss,us_real_gdp,co2
0,2001/1/7,27.80,78.645985,96641.56859,9.634759e+05,283960.0,5944,1932323.219,12999.5699,370.30
1,2001/1/14,28.81,68.158536,89172.96037,9.010023e+05,284011.5,5949,2174659.188,12293.3405,370.67
2,2001/1/21,30.63,76.149426,100533.68770,1.032450e+06,284063.0,5912,2335532.223,12708.8104,370.52
3,2001/1/28,31.35,76.116224,101753.62190,8.260147e+05,284114.5,5935,2065545.899,12287.2265,371.36
4,2001/2/4,29.59,66.693151,61448.50321,7.766296e+05,284166.0,5929,1989475.501,13353.2628,371.58
...,...,...,...,...,...,...,...,...,...,...
412,2008/11/30,53.27,99.238162,93144.23857,4.333457e+05,305785.6,5118,2254337.427,15451.9032,385.05
413,2008/12/7,45.60,82.367875,93097.53674,9.989935e+05,305827.0,5152,2031384.867,15362.3704,385.29
414,2008/12/14,44.57,96.265108,96031.18996,5.635120e+05,305879.0,4967,2397440.218,14770.9146,385.54
415,2008/12/21,39.70,92.911057,94626.38776,7.542331e+05,305931.0,4972,2535053.244,15966.6992,385.93


# Creating independent and dependent variables 

In [3]:
# calculate independent variables
periods = (1,2,3,5,10)
for i in periods:
    df['of_chg_{}'.format(i)] = df['crude_oil'].pct_change(periods = i)
    
# calculate dependent variable
df['co2_chg'] = df['co2'].pct_change(periods=1)

# eliminate the empty rows
df = df[11:]
df

,week,crude_oil,cpi,coal_consumption,ngas_consumption,us_pop,oil_production,tree_cover_loss,us_real_gdp,co2,of_chg_1,of_chg_2,of_chg_3,of_chg_5,of_chg_10,co2_chg
11,2001/3/25,26.42,71.870113,74962.95365,624013.0521,284546.5,5868,1728742.189,11982.2289,372.40,-0.022206,-0.071353,-0.053386,-0.109538,-0.082957,-0.000376
12,2001/4/1,26.86,74.862456,90444.57488,614474.9072,284602.0,5883,2248741.369,13936.6979,372.45,0.016654,-0.005922,-0.055888,-0.062478,-0.123082,0.000134
13,2001/4/8,26.76,67.840486,71861.08162,436068.6622,284648.4,5867,1724128.479,12809.3245,373.61,-0.003723,0.012869,-0.009623,-0.041204,-0.146411,0.003115
14,2001/4/15,28.27,82.229017,76914.86811,449159.7901,284694.8,5855,1956950.159,13474.2006,373.71,0.056428,0.052494,0.070023,-0.006327,-0.044610,0.000268
15,2001/4/22,27.89,76.824931,73912.39968,-37896.8606,284741.2,5861,1366131.633,13236.2220,373.60,-0.013442,0.042227,0.038347,0.032198,-0.097995,-0.000294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,2008/11/30,53.27,99.238162,93144.23857,433345.7100,305785.6,5118,2254337.427,15451.9032,385.05,0.019326,-0.090956,-0.171668,-0.223016,-0.451898,0.001456
413,2008/12/7,45.60,82.367875,93097.53674,998993.4526,305827.0,5152,2031384.867,15362.3704,385.29,-0.143983,-0.127440,-0.221843,-0.300721,-0.589633,0.000623
414,2008/12/14,44.57,96.265108,96031.18996,563512.0399,305879.0,4967,2397440.218,14770.9146,385.54,-0.022588,-0.163319,-0.147149,-0.306951,-0.538565,0.000649
415,2008/12/21,39.70,92.911057,94626.38776,754233.0514,305931.0,4972,2535053.244,15966.6992,385.93,-0.109266,-0.129386,-0.254740,-0.322526,-0.539657,0.001012


# Seperating training, validation, and test datasets

In [4]:
# create list of independent variable's name TODO: add the appropriate variable names to the list
ind_var_name = ['of_chg_{}'.format(i) for i in periods]+['cpi','coal_consumption','ngas_consumption',
                                                         'us_pop','oil_production','tree_cover_loss','us_real_gdp']
num_var =len(ind_var_name) # put number of variables here

# extract the values to X
X = df[['crude_oil'] + ind_var_name].values
X

array([[ 2.64200000e+01, -2.22057735e-02, -7.13532513e-02, ...,
         5.86800000e+03,  1.72874219e+06,  1.19822289e+04],
       [ 2.68600000e+01,  1.66540500e-02, -5.92153960e-03, ...,
         5.88300000e+03,  2.24874137e+06,  1.39366979e+04],
       [ 2.67600000e+01, -3.72300819e-03,  1.28690386e-02, ...,
         5.86700000e+03,  1.72412848e+06,  1.28093245e+04],
       ...,
       [ 4.45700000e+01, -2.25877193e-02, -1.63318941e-01, ...,
         4.96700000e+03,  2.39744022e+06,  1.47709146e+04],
       [ 3.97000000e+01, -1.09266323e-01, -1.29385965e-01, ...,
         4.97200000e+03,  2.53505324e+06,  1.59666992e+04],
       [ 3.29800000e+01, -1.69269521e-01, -2.60040386e-01, ...,
         4.97900000e+03,  2.04491061e+06,  1.53330495e+04]])

In [5]:
# create y for regressor, y_c for classifier
y = df['co2_chg']
y_c = (df['co2_chg'] > 0).values.astype('int')

# set training, validation, and test criterias
training_ratio = 0.7
validation_ratio = 0.15
test_ratio = 0.15

# check correctness of X and y
X.shape, y.shape, y_c.shape

((406, 13), (406,), (406,))

In [6]:
# create training, validation, and test sets for regressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_ratio, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train shape:\t", X_train.shape)
print("X_test shape:\t", X_test.shape)
print("X_val shape:\t", X_val.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train shape:	 (284, 13)
X_test shape:	 (61, 13)
X_val shape:	 (61, 13)
y_train shape:	 (284,)
y_val shape:	 (61,)
y_test shape:	 (61,)


In [7]:
# create training, validation, and test sets for classifier
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X, y_c, test_size = test_ratio, random_state = 42)
X_train_c, X_val_c, y_train_c, y_val_c = train_test_split(X_train_c, y_train_c, test_size = validation_ratio/(validation_ratio + training_ratio), random_state = 42)
print("X_train_c shape:", X_train_c.shape)
print("X_test_c shape:\t", X_test_c.shape)
print("X_val_c shape:\t", X_val_c.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c shape: (284, 13)
X_test_c shape:	 (61, 13)
X_val_c shape:	 (61, 13)
y_train_c shape: (284,)
y_val_c shape:	 (61,)
y_test_c shape:	 (61,)


# Principal Component Analysis (PCA)

In [8]:
# normalization for regressor set
for i in range(X_train.shape[1]):
    X_train_mean = X_train[:, i]. mean()
    X_train_std = X_train[:, i].std()
    X_train[:, i] = (X_train[:,i] - X_train_mean) / X_train_std
    X_test[:,i] = (X_test[:,i] - X_train_mean) / X_train_std
    X_val[:, i] = (X_val[:,i] - X_train_mean) / X_train_std

print("X_train shape:\t", X_train.shape)
print("X_val shape:\t", X_val.shape)

X_train shape:	 (284, 13)
X_val shape:	 (61, 13)


In [9]:
# normalization for classifier set
for i in range(X_train_c.shape[1]):
    X_train_c_mean = X_train_c[:, i]. mean()
    X_train_c_std = X_train_c[:, i].std()
    X_train_c[:, i] = (X_train_c[:,i] - X_train_c_mean) / X_train_c_std
    X_test_c[:,i] = (X_test_c[:,i] - X_train_c_mean) / X_train_c_std
    X_val_c[:, i] = (X_val_c[:,i] - X_train_c_mean) / X_train_c_std

print("X_train_c shape:", X_train_c.shape)
print("X_val_c shape:\t", X_val_c.shape)

X_train_c shape: (284, 13)
X_val_c shape:	 (61, 13)


In [10]:
# PCA for regressor
# fit on training set
scaler = StandardScaler()
scaler.fit(X_train)

# apply transform to training and test set
X_train_pca = scaler.transform(X_train)
X_test_pca = scaler.transform(X_test)
X_val_pca = scaler.transform(X_val)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_pca)

# apply transform to training and test set
X_train_pca = pca.transform(X_train_pca)
X_test_pca = pca.transform(X_test_pca)
X_val_pca = pca.transform(X_val_pca)
print("X_train_pca shape:\t", X_train_pca.shape)
print("X_test_pca shape:\t", X_test_pca.shape)
print("X_val_pca shape:\t", X_val_pca.shape)
print("y_train shape:\t", y_train.shape)
print("y_val shape:\t", y_val.shape)
print("y_test shape:\t", y_test.shape)

X_train_pca shape:	 (284, 9)
X_test_pca shape:	 (61, 9)
X_val_pca shape:	 (61, 9)
y_train shape:	 (284,)
y_val shape:	 (61,)
y_test shape:	 (61,)


In [11]:
# PCA for classifier
# fit on training set_c
scaler = StandardScaler()
scaler.fit(X_train_c)

# apply transform to training and test set
X_train_c_pca = scaler.transform(X_train_c)
X_test_c_pca = scaler.transform(X_test_c)
X_val_c_pca = scaler.transform(X_val_c)

# set PCA value
pca = PCA(0.95)

# fit PCA training set
pca.fit(X_train_c_pca)

# apply transform to training and test set
X_train_c_pca = pca.transform(X_train_c_pca)
X_test_c_pca = pca.transform(X_test_c_pca)
X_val_c_pca = pca.transform(X_val_c_pca)
print("X_train_c_pca shape:\t", X_train_c_pca.shape)
print("X_test_c_pca shape:\t", X_test_c_pca.shape)
print("X_val_c_pca shape:\t", X_val_c_pca.shape)
print("y_train_c shape:", y_train_c.shape)
print("y_val_c shape:\t", y_val_c.shape)
print("y_test_c shape:\t", y_test_c.shape)

X_train_c_pca shape:	 (284, 9)
X_test_c_pca shape:	 (61, 9)
X_val_c_pca shape:	 (61, 9)
y_train_c shape: (284,)
y_val_c shape:	 (61,)
y_test_c shape:	 (61,)


In [12]:
num_var = X_train_c_pca.shape[1]
num_var

9

# Multiple linear regression

In [13]:
multiple_reg = LinearRegression().fit(X_train_pca, y_train)
score_multiple_reg = multiple_reg.score(X_test_pca, y_test)
print('score_multiple_reg:', "%.4f"%score_multiple_reg)

score_multiple_reg: 0.0477


# Multiple polynomial regression

In [14]:
#to prove this is the correct polynomial regression, set degree to 1
poly = PolynomialFeatures(1)
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

0.04767658762467197

In [15]:
# use val to find the appropriate degree
mpr = dict()
for i in range(2,6):
    degree=i
    poly = PolynomialFeatures(degree)
    # polynomial transform
    X_train_poly = poly.fit_transform(X_train_pca, y_train)
    X_val_poly = poly.fit_transform(X_val_pca,y_val)
    X_test_poly = poly.fit_transform(X_test_pca,y_test)

    # fit to linear regression
    polyreg = LinearRegression().fit(X_train_poly, y_train)

    # predict using liear regression
    score_poly_reg = polyreg.score(X_val_poly, y_val)
    print('score_poly_reg, degree',i,':', "%.4f"%score_poly_reg)
    mpr[f'{i}'] = score_poly_reg
    
max_degree = max(mpr, key = mpr.get)
print('max degree:', max_degree)

score_poly_reg, degree 2 : -0.0903
score_poly_reg, degree 3 : -16.1756
score_poly_reg, degree 4 : -37.6736
score_poly_reg, degree 5 : -159.7892
max degree: 2


In [16]:
poly = PolynomialFeatures(int(max_degree))
# polynomial transform
X_train_poly = poly.fit_transform(X_train_pca, y_train)
X_val_poly = poly.fit_transform(X_val_pca,y_val)
X_test_poly = poly.fit_transform(X_test_pca,y_test)

# fit to linear regression
polyreg = LinearRegression().fit(X_train_poly, y_train)

# predict using liear regression
score_poly_reg = polyreg.score(X_test_poly, y_test)
score_poly_reg

-0.04689632794147669

# Random forest classifier

In [17]:
# testing for different max depth
rfc = dict()
for i in range (1, 15):
    clf = RandomForestClassifier(max_depth=i, random_state=0).fit(X_train_c_pca, y_train_c)
    y_pred = clf.predict(X_val_c_pca)
    score_clf = accuracy_score(y_val_c, y_pred)
    rfc[f'{i}'] = score_clf
    #print('score_clf_',i,":", "%.4f"%score_clf)
max_depth = max(rfc, key=rfc.get)
print('max depth:', max_depth)

max depth: 2


In [18]:
# choose max_depth = 4
clf = RandomForestClassifier(max_depth=int(max_depth), random_state=0).fit(X_train_c_pca, y_train_c)
score_rf_clf = clf.score(X_test_c_pca, y_test_c)
print('score_rf_clf:', "%.4f"%score_rf_clf)

score_rf_clf: 0.6885


# Random forest regressor

In [19]:
clf_reg = RandomForestRegressor().fit(X_train_pca, y_train)
score_rf_reg = clf_reg.score(X_test_pca, y_test)
print('score_rf_reg:', "%.4f"%score_rf_reg)

score_rf_reg: 0.1762


# Logistic regression

In [20]:
logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear', random_state=0).fit(X_train_c_pca, y_train_c)
score_logistic = logistic_model.score(X_test_c_pca, y_test_c)
print('score_logistic:', "%.4f"%score_logistic)

score_logistic: 0.5246


# Gradient boosting classifier

In [21]:
gra_clf = GradientBoostingClassifier(n_estimators = 200, learning_rate=0.50, max_depth=5, random_state=0).fit(X_train_c_pca, y_train_c)
score_gb_clf = gra_clf.score(X_test_c_pca, y_test_c)
score_gb_clf

0.6557377049180327

# Gradient boosting regressor 

In [22]:
gra_reg = GradientBoostingRegressor(random_state=0).fit(X_train_pca, y_train)
score_gb_reg = gra_reg.score(X_test_pca, y_test)
score_gb_reg

0.08328240517724594

# XGBoost classifier

In [23]:
xgb_clf = XGBClassifier().fit(X_train_c_pca, y_train_c)
score_xgb_clf = xgb_clf.score(X_test_c_pca, y_test_c)
score_xgb_clf

[17:15:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\lemon\anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.6721311475409836

# XGBoost regressor

In [24]:
xgb_reg = XGBRegressor(random_state = 0).fit(X_train_pca, y_train)
score_xgb_reg = xgb_reg.score(X_test_pca, y_test)
score_xgb_reg

0.08074264527308361

# AdaBoost classifier

In [25]:
adb_clf = AdaBoostClassifier().fit(X_train_c_pca, y_train_c)
score_adb_clf = adb_clf.score(X_test_c_pca, y_test_c)
score_adb_clf

0.6065573770491803

# AdaBoost regressor

In [26]:
adb_reg = AdaBoostRegressor().fit(X_train_pca, y_train)
score_adb_reg = adb_reg.score(X_test_pca, y_test)
score_adb_reg

0.041222107446489575

# Support vector classification

In [27]:
# Try different kernels
kernels = ('linear', 'poly', 'rbf')

for k in kernels:
    svm_clf = SVC(kernel=k).fit(X_train_c_pca, y_train_c)
    y_pred = svm_clf.predict(X_test_c_pca)
    vars()[f'score_svc_{k}']= accuracy_score(y_test_c, y_pred)
    
print(score_svc_linear)
print(score_svc_poly)
print(score_svc_rbf)

0.5737704918032787
0.5081967213114754
0.5901639344262295


# Gaussian Naive Bayes

In [28]:
gnb_clf = GaussianNB().fit(X_train_c_pca, y_train_c)
score_gnb = gnb_clf.score(X_test_c_pca, y_test_c)
print(score_gnb)

0.6885245901639344


# Neuro networks

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(10, input_dim=num_var, activation='relu'))
model.add(keras.layers.Dense(5, activation='relu'))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_c_pca, y_train_c, epochs=50, validation_data=(X_val_c_pca, y_val_c))

Epoch 1/50
9/9 [==============================] - 2s 137ms/step - loss: 0.6920 - accuracy: 0.4804 - val_loss: 0.7141 - val_accuracy: 0.4590
Epoch 2/50
9/9 [==============================] - 0s 6ms/step - loss: 0.6814 - accuracy: 0.5287 - val_loss: 0.7116 - val_accuracy: 0.4918
Epoch 3/50
9/9 [==============================] - 0s 5ms/step - loss: 0.6688 - accuracy: 0.5535 - val_loss: 0.7092 - val_accuracy: 0.4918
Epoch 4/50
9/9 [==============================] - 0s 5ms/step - loss: 0.6761 - accuracy: 0.5397 - val_loss: 0.7073 - val_accuracy: 0.5082
Epoch 5/50
9/9 [==============================] - 0s 6ms/step - loss: 0.6807 - accuracy: 0.5088 - val_loss: 0.7057 - val_accuracy: 0.5082
Epoch 6/50
9/9 [==============================] - 0s 5ms/step - loss: 0.6732 - accuracy: 0.5492 - val_loss: 0.7042 - val_accuracy: 0.5082
Epoch 7/50
9/9 [==============================] - 0s 6ms/step - loss: 0.6781 - accuracy: 0.5368 - val_loss: 0.7029 - val_accuracy: 0.5082
Epoch 8/50
9/9 [================

In [30]:
loss, score_neuro = model.evaluate(X_test_c_pca, y_test_c)
print(score_neuro)

2/2 [==============================] - 0s 998us/step - loss: 0.7077 - accuracy: 0.5082
0.5081967115402222


# Summary

In [31]:
reg = {
    'Multiple linear regression': score_multiple_reg,
    'Multiple polynomial regression': score_poly_reg,
    'Random forest regressor': score_rf_reg,
    'Gradient boosting regressor': score_gb_reg,
    'XGBoost regressor': score_xgb_reg,
    'AdaBoost regressor': score_adb_reg
}
reg_df = pd.DataFrame.from_dict(reg, orient='index', columns = ['Score'])
reg_df.index.name = 'Regression models'
#reg_df.sort_values(by=['Score'], ascending=False)
reg_df

,Score
Regression models,
Multiple linear regression,0.047677
Multiple polynomial regression,-0.046896
Random forest regressor,0.176172
Gradient boosting regressor,0.083282
XGBoost regressor,0.080743
AdaBoost regressor,0.041222


In [32]:
clf = {
    'Logistic regression': score_logistic,
    'Random forest classifier': score_rf_clf,
    'Gradient boosting classifier': score_gb_clf,
    'XGBoost classifier': score_xgb_clf,
    'AdaBoost classifier': score_adb_clf,
    'Gaussian naive Bayes': score_gnb, 
    'SVC linear': score_svc_linear,
    'SVC polynomial': score_svc_poly, 
    'SVC RBF': score_svc_rbf,
    'Neuro network' : score_neuro
}
clf_df = pd.DataFrame.from_dict(clf, orient='index',columns = ['Score'])
clf_df.index.name = 'Classifier models'
#clf_df.sort_values(by=['Score'], ascending=False)
clf_df

,Score
Classifier models,
Logistic regression,0.524590
Random forest classifier,0.688525
Gradient boosting classifier,0.655738
XGBoost classifier,0.672131
AdaBoost classifier,0.606557
Gaussian naive Bayes,0.688525
SVC linear,0.573770
SVC polynomial,0.508197
SVC RBF,0.590164
